# Lab 4: Frequency Response of an RC Filter

**Note:** This lab procedure is kind of long with some explanatory material.  I have added the table of contents below to help you navigate the document.  I have also added links in various places to jump back to the TOC: [back to TOC](#toc).

**Note 2:** Processing the fixed sine data may take a little time.  You might want to process that data last to make sure you collect all of the data you need.

<a id='toc'></a>

## Table of Contents

- [Introduction](#intro)
- [software update](#sw_update)
- [simulation of fixed sine input for an RC filter](#fixed_sim)
- [experimental fixed sine testing](#fixed_exp)
- [experimental swept sine testing](#swept_exp)
- [Bode plot generation](#bode_gen)
- [comprehension questions](#compq)

<a id='intro'></a>

## Introduction

In dynamic systems, frequency response refers to investigating how a system will response to fixed sine inputs of various frequencies.  As the frequency of the input sine wave changes, the magnitude of the output and the phase difference between the input and output will also change. The results of frequency response testing over a wide range of frequencies can be shown on a Bode plot, which is a powerful tool for experimental system identification.

## Three Main Phases

This lab consists of three main phases:

- [simulation of fixed sine input for an RC filter](#fixed_sim)
- [experimental fixed sine testing](#fixed_exp)
- [experimental swept sine testing](#swept_exp)

## Background

If a sinusoidal input is given to a stable dynamic system, after transients die out the output will be a sine wave at the same frequency as the input.  However, the amplitude of the output may be larger or smaller than the amplitude of the input and there may be a phase difference between the two.  Plotting the magnitude ratio and the phase versus input frequency can tell us a lot about the system.

[back to TOC](#toc)

<a id='sw_update'></a>

## Software Update

Shockingly, I don't think you need to update the software this week.

<a id='fixed_sim'></a>

## Fixed Sine Simulations

In order to understand what data to expect from your experiments, first run a series of simulated fixed sine responses on an RC filter TF.  

$$G(s) = \frac{p}{s+p}$$

Set `p = 5*2*pi` (i.e. 5 Hz) and then use `control.forced_response` to find the response to several different input frequencies:

`u = np.sin(2*pi*f*t)` 

where `f = [1, 3, 5, 7, 10, 25]` Hz.  For each input frequency, plot the input and output on one plot:

<img src="https://drive.google.com/uc?id=1tKJ4qZtzPsyzD_gqCnRemjKDOuA5yTmK" width=500px>

### Fixed Sine Simulation Observations

- What happens to the magnitude and phase as the input frequency increases?

[back to TOC](#toc)

<a id='fixed_exp'></a>

## Fixed Sine Experiments

PWM cannot support a negative duty cycle (i.e. on-time cannot be less than 0).  In order to perform sinusoidal testing, we will need to add an offset to shift the baseline to 2.5V (50% duty cycle).  This is done by creating an `int_constant_block` with a value of 128 and then adding this to the sinsoidal input like the one shown below:

<img src="https://drive.google.com/uc?id=1tZIG6QkLbKN_D_PQXWXCeeH-UmIjgSwk" width=500px>

A `fixed_sine_input` block generates a sine wave signal with a fixed frequency:

$$u(t) = A \sin(2\pi f t)$$

where $A$ is an amplitude in counts and $f$ is the frequency in Hz.

Once you have the sinusoidal input, constant offset, and add blocks, the rest is the same as lab two: `i2c_actuator`, `i2c_sensor`, and `i2c_plant`.

### Arduino Code

Make sure you have programmed your Arduino using the code `arduino_i2c_siso_RC_circuit_v2.ino`.  There are a few changes from the lab 2 code that make the data a little cleaner.

### Fixed Sine Data Collection

Your goal is to save 6 fixed sine tests to csv files.  Each test is run at a different fixed sine frequency.  The frequencies you choose are based on a desired magnitude ratio and phase shift, which are defined in the next section.

1. really low frequency
    - magnitude ratio nearly 1
    - minimal phase shift
    - **make sure the really low frequency data looks good before collecting the rest of the data**
        - you will need to shift and scale your output data because `analogRead` is 10 bit and `analogWrite` is 8 bit on an Arduino
2. low frequency
    - magnitude ratio slightly less than 1
    - small amount of phase shift
3. slightly below `p`
    - magnitude ratio approximately 0.7
4. slightly above `p`
    - magnitude ratio approximately 0.5
5. fairly high frequency
    - magnitude ratio getting small
    - phase shift of 70-80$^\circ$
6. pretty high frequency
    - phase shift approaching 90$^\circ$
    
**Note:** 50 Hz is about the maximum we can do with the Arduino settings.

### Fixed Sine Data Processing: Magnitude Ratio and Phase 

For each fixed sine input frequency, estimate the amplitude ratio and the phase difference.  The magintude ratio is the amplitude of the output sine wave divided by the amplitude of the input sine wave:

$$Mag. \; Ratio = \frac{Amp_{out}}{Amp_{in}}$$

<img src="https://drive.google.com/uc?id=1T4raNLhL0RvlILKFjOdu4mG9OLGKZdHZ" width=500px>

Estimating the phase lag is slightly trickier.  You need to find the difference in peak times or zero crossing times between the input and ouput and then compare that to the period.  A full period is 360$^\circ$:

$$phase = \frac{T_{diff}}{T_{period}} \cdot 360^\circ$$

where $T_{diff}$ is the difference in peak times or zero crossing times.  $T_{diff}$ is typically negative.  Here is a picture illustrating $T_{diff}$:

<img src="https://drive.google.com/uc?id=1SzoWKPt6LMEae4Pd0t9IMnOKMbOZHW6W" width=500px>

## Frequency Response Plot

Plot your magnitude ratios and phases from the fixed sine data on two separate $y$ axes with frequency on the $x$ axis. 

[back to TOC](#toc)

<a id='swept_exp'></a>

## Experimental Swept Sine Testing

### Generating a Swept Sine Input Signal

Fixed sine inputs help us grow in our understanding of frequency response, but
running tests at many fixed frequencies is time consuming.  In a fixed sine test

$$u(t) = A \sin(2 \pi f t)$$

where $f$ was a constant.

In swept sine testing, $f$ is a function of time:

$$f(t) = m t$$

where $m$ is a slope based on the maximum desired frequency and the total time the test will be run:

$$m = \frac{f_{max}}{t_{max}}$$

So, for a swept sine input:

$$u_{ss}(t) = A \sin(2 \pi f(t) \cdot t)$$

Here is an example of a swept sine signal:

<img src="https://drive.google.com/uc?id=1gbRi4A-H732VHhDEzUQsgD3JdzhNeO_2" width=500px>


### Running a Swept Sine Test

The `pybd_gui` should have a `swept_sine_input` block (once you have updated). The parameters for the block are the amplitude and the slope.  Also note that the menu parameters dialog now includes global parameters.  For now `stop_t` is the only global parameter.  `stop_t` represents the total time a swept sine test should run.  Note that it may take some trial and error to find "good" values for slope and `stop_t`.  Note that sometimes swept sine tests need to run for 10 seconds or more to get "good" data.  Examples of good and bad data are in the next section.

### Possible Issues in Swept Sine Testing

#### Picking a "good" amplitude

This will be more of an issue once you attach the beam, but picking the amplitude for your input can be fairly important.  Just like with fixed sine testing, this might be a trial and error process.  If your amplitude is too large, you may cause saturation or break something in the physical system (such as the beam).  This is tricky because the output amplitude will change as a function of frequency. 

#### Slope too high

Another important part of swept sine testing is running the test for an appropriate amount of time with an appropriate frequency slope.  Ideally, the output should track the input at low frequencies, but the test should continue until the output has a hard time keeping up at high frequencies.

Here is an example of a slope that is too high:

<img src="https://drive.google.com/uc?id=1gjAsElnB96WljsIi3gPfNM1KVn6qpxnK" width=500px>

Note that the output is not able to keep up with the input for very much of the test.

#### Slope too low or max T too small

Here is an example of a slope that is too small or of a test that was not run for long enough:

<img src="https://drive.google.com/uc?id=1gdme-EsASPiP3PUoHYs3dEjyS6APOKde" width=500px>

Note that the output is starting to get smaller, but it is still somewhat keeping up.

#### Good slope example

Here is an example of a pretty good slope choice:

<img src="https://drive.google.com/uc?id=1h2-a-Usbw9HrEcKNIWvb8mQJjA-sNyBk" width=500px>

Note that the output is able to track the input for the first part of the test, but the output has mostly died out by the end of the test.

[back to TOC](#toc)

<a id='bode_gen'></a>

## Generating a Bode Plot from Experimental Data

Once you have "good" swept-sine data for the input and output of the system, you are ready to calculate $G(j\omega)$.  To do this, we will use an `fft` algorithm provided by the `numpy` module (`numpy.fft.fft`).  To avoid typing all of that, we will import it as `fft`:

In [1]:
from numpy.fft import fft

`fft` stands for fast Fourier transform.  We don't have time to go into too many details about it right now.  The main thing you need to know is that:

$$G(j\omega) = \frac{\texttt{fft(output)}}{\texttt{fft(input)}}$$

Once we have $G(j\omega)$ from the experimental data, we are ready to calculate the dB magnitude and phase.  The final step in generating a Bode plot from experimental data is finding the frequency vector.  If we know the TF and want to generate a Bode plot we can just pick the frequency vector (just like picking a time vector when generating a plot).  When working with experimental data, we need to calculate the frequency vector that corresponds to our time domain data.  If data is collected for a certain length of time $T$ with a certain sampling frequency $f_s$, then

$$dt = \frac{1}{f_s}$$

and

$$df = \frac{1}{T}$$

We ran our tests at 500 Hz, so that $f_s = 500$ Hz and $dt = 0.002$ seconds.

- $f_s$ is controlled by the Arduino timer settings.

Our time vector will be

$$t = \left[0, dt, 2 \cdot dt, 3 \cdot dt, ..., T-dt\right]$$

And the frequency vector will be

$$freq = \left[0, df, 2 \cdot df, 3 \cdot df, ..., f_s-df\right]$$

The easiest way to find the frequency vector in Python is to first calculate $df$.  Then, create an `nvect` based on the length of `t`:

```
N = len(t)
nvect = np.arange(N)
```

Then, the frequency vector will be

```
freq = df*nvect
```

### Fixed Sine vs. Swept Sine

Generate a Bode plot that overlays the fixed sine frequency response points you found doing fixed sine testing with the Bode plot from the swept sine test that you just ran.

[back to TOC](#toc)

<a id='compq'></a>

## Comprehension Questions

### Comp. Q1

<img src="https://drive.google.com/uc?id=1T_MYkK7uJQJ2kgZb17WcUmomQHQcfiJr" width=500px>

**Q1:** Why would it not be correct to use the two points marked by the green squares in the graph above to calculate magnitude ratio and phase for a fixed sine frequency response test?

### Comp. Q2

<img src="https://drive.google.com/uc?id=17ZLIk_4g5fVOX0b8Po5hWpbYhJfOt35M" width=500px>

**Q2:** Why would it not be correct to use the two points marked by the green triangles in the graph above to calculate magnitude ratio and phase for a fixed sine frequency response test?

[back to TOC](#toc)